# Thermal Design

## Tools

In [16]:
from math import sqrt, pi

def circle_packing_hexagonal(radious, width, height):
    """ ref: https://math.stackexchange.com/a/2548599/897111
        height = ((2 * rows) +1) * radious
        width = (2 + (cols - 1) * sqrt(3)) * radious """
    
    rows = int(((height / radious) -1) / 2)
    cols = int((((width / radious) -2) / sqrt(3)) + 1)
    circles = rows * cols
    
    rectangle_area = width * height
    circle_area = pi * radious**2
    hexagonal_packing_density = (pi * sqrt(3) / 6)
    packing_efficiency = (circle_area * circles) / (hexagonal_packing_density * rectangle_area)
    
    return rows, cols, circles, packing_efficiency

width = 257
height = 157
radious = sqrt(10 / pi)
rows, cols, circles, packing_efficiency = circle_packing_hexagonal(radious, height, width)

print('rows:', rows)
print('cols:', cols)
print('circles:', circles)
print('packing_efficiency:', packing_efficiency)

rows: 71
cols: 50
circles: 3550
packing_efficiency: 0.9701442805260908


## Design
References:
- [Nexperia - The Power MOSFET Application Handbook: Design Engineer's Guide](https://www.mouser.com/pdfdocs/Nexperia_MOSFET_Handbook.pdf)
- [ONSemi - Guide to Thermally Enhanced SO8-FL](https://www.onsemi.com/pub/Collateral/AND9014-D.PDF)
- [Sound-au - The Design of Heatsinks](https://sound-au.com/heatsinks.htm)
- [PROPRIEDADES TÉRMICAS DE MATERIAIS](http://www.foz.unioeste.br/~lamat/downmateriais/materiaiscap13.pdf)
- [Nexperia - AN90003 - LFPAK MOSFET thermal design guide](https://assets.nexperia.com/documents/application-note/AN90003.pdf)
- [ST - AN1703 - GUIDELINES FOR USING ST’S MOSFET SMD PACKAGES](https://www.st.com/resource/en/application_note/cd00004438-guidelines-for-using-sts-mosfet-smd-packages-stmicroelectronics.pdf)
- [TI - Thermal Considerations for Designing a GaN Power Stage](https://www.ti.com/lit/an/snoaa14b/snoaa14b.pdf)
- [TI - Thermal Comparison of FR-4 and Insulated Metal Substrate PCB for GaN Inverter](https://www.ti.com/lit/an/tida030/tida030.pdf)
- [TI - AN-2020 Thermal Design By Insight, Not Hindsight](https://www.ti.com/lit/an/snva419c/snva419c.pdf)
- [Infineon - 600 V CoolMOS™ C7 Gold (G7) - Chapter 4: Thermal handling](https://www.infineon.com/dgdl/Infineon-ApplicationNote_600V_CoolMOS_C7_Gold_TOLL-AN-v01_00-EN.pdf)
- [Sil-Pad® Design Guide](http://www.farnell.com/datasheets/99609.pdf)
- [Implastec - THERMAL PAD – Adesivo Termo-Condutivo](https://implastec.com.br/loja/isolador-disipador-silglass/)
- [Thermal Interface Materials for Power Electronics Applications](https://www.nrel.gov/docs/fy08osti/42972.pdf)
- [Thermal Paste Application](https://www.semikron.com/dl/service-support/downloads/download/semikron-application-note-thermal-paste-application-en-2018-01-19-rev-00/)

In [40]:
from math import sqrt, pi, ceil

k = { # Thermal conductivity [W/(m*K)]
    'Copper': 385,
    'Aluminium': 205,
    'Solder': 50,  # typical 50µm
    'Alumina': 25,  # Implastec: 19.5 x 13.5 x 0.6 mm | 22 x 16 x 0.6 mm
    'Mica': 1.7,  # Implastec: 19.5 x 13.5 x 0.2 mm | 22 x 16 x 0.2 mm
    'FR-4': 0.25,
    'Implastec Thermal Silver': 1.6,  # typical from 20µm to 80µm
    'Implastec Céramique': 1.2,  # typical from 20µm to 80µm
    'Implastec IPT': 0.4,  # typical from 20µm to 80µm
    'Implastec Thermal Pad (Silglass)': 0.9,  # 0.178mm
    'Air': 0.024,
}

def compute_via_resistance(conductivity, drill_diameter, pcb_thickness, plating_thickness):
    """ drill_diameter is 0.15mm minimal, 0.3mm typical
        drill_diameter is 0.4mm minimal, 1.6mm typical
        plating_thickness is 20µm for typical ICP class I and II, 35µm for IPC Class III """
    external_radious = drill_diameter / 2
    internal_radious = external_radious - plating_thickness
    area = pi * (external_radious**2 - internal_radious**2)
    print('Via cross-section area:', area)
    return pcb_thickness / (conductivity * area)

def compute_cuboid_resistance(conductivity, height, width, thickness):
    area = (height * width)
    return thickness / (conductivity * area)

print('Stack: MOSFET -> Solder -> Copper -> Via -> Copper -> TIM -> HeatSink -> Ambient')
print()

# Mosfet PAD Area
pad_height, pad_width = 8e-3, 10e-3
pad_area = pad_width * pad_height
print('MOSFET Pad Height:', pad_height, 'm')
print('MOSFET Pad Width:', pad_width, 'm')
print('MOSFET Pad Area:', pad_area, 'm²')

# MOSFET Solder to PCB
solder_thickness = 0.05e-3
solder_material = 'Solder'
solder_resistance = compute_cuboid_resistance(
    k[solder_material], pad_height, pad_width, solder_thickness)
print('Solder Material:', solder_material)
print('Solder Material Conductivity:', k[solder_material], 'W/mK')
print('Solder Thickness:', solder_thickness, 'm')
print('Solder Resistance:', solder_resistance, '°C/W')
print()

# PCB Copper Layer
copper_thickness = 35e-6  # 1oz: 35µm
copper_material = 'Copper'
copper_resistance = compute_cuboid_resistance(
    k[copper_material], pad_height, pad_width, copper_thickness)
print('Copper Layer Material:', copper_material)
print('Copper Layer Material Conductivity:', k[copper_material], 'W/mK')
print('Copper Layer Thickness:', copper_thickness, 'm')
print('Copper Layer Resistance:', copper_resistance, '°C/W')
print()

# PCB Vias
via_plating_thickness = 20e-6  # Level 2 standard = 20µm, Level 3 standard: 35µm
drill_diameter = 0.3e-3
via_clearance = drill_diameter / sqrt(2)
pcb_thickness = 0.6e-3
via_radious = (drill_diameter / 2) + via_clearance
print('PCB Thickness:', pcb_thickness, 'm')
print('Via Plating Thickness:', via_plating_thickness, 'm')
print('Via Drill Diameter:', drill_diameter, 'm')
print('Via Clearance:', via_clearance, 'm')
print('Via Radious:', via_radious, 'm')

nrows, ncols, nvias, packing_efficiency = circle_packing_hexagonal(
    via_radious, pad_height, pad_width)
print('\tRows of Vias:', nrows)
print('\tColumns of Vias:', ncols)
print('\tNumber of Vias:', nvias)
print('\tArea Utilization:', packing_efficiency)

via_material = 'Copper'
print('Via Material:', via_material)
print('Via Material Conductivity:', k[via_material], 'W/mK')
single_via_resistance = compute_via_resistance(
    k[via_material], drill_diameter, pcb_thickness, via_plating_thickness)
via_resistance = single_via_resistance / nvias
print('Single Via Resistance:', single_via_resistance, '°C/W')
print('Array of Vias Resistance:', via_resistance, '°C/W')
print()

# Bus Bar
# busbar_thickness = 5e-3
# busbar_material = 'Copper'
# busbar_resistance = compute_cuboid_resistance(k[busbar_material], pad_height, pad_width, busbar_thickness)
# print('Bus Bar Material:', busbar_material)
# print('Bus Bar Material Conductivity:', k[busbar_material], 'W/mK')
# print('Bus Bar Thickness:', busbar_thickness, 'm')
# print('Bus Bar Resistance:', busbar_resistance, '°C/W')
# print()

# Thermal Interface Material (Electrical Insulator)
tim_thickness = 178e-6
tim_material = 'Implastec Thermal Pad (Silglass)'
# tim_thickness = 0.2e-3
# tim_material = 'Mica'
tim_resistance = compute_cuboid_resistance(
    k[tim_material], pad_height, pad_width, tim_thickness)
print('TIM Material:', tim_material)
print('TIM Material Conductivity:', k[tim_material], 'W/mK')
print('TIM Thickness:', tim_thickness, 'm')
print('TIM Resistance:', tim_resistance, '°C/W')
print()

# Total Stack Thermal Resistance
total_resistance = 0
total_resistance += solder_resistance
total_resistance += copper_resistance
total_resistance += via_resistance
total_resistance += copper_resistance
total_resistance += tim_resistance
print('Total Resistance between MOSFET and HeatSink:', total_resistance, '°C/W')
print()


Stack: MOSFET -> Solder -> Copper -> Via -> Copper -> TIM -> HeatSink -> Ambient

MOSFET Pad Height: 0.008 m
MOSFET Pad Width: 0.01 m
MOSFET Pad Area: 8e-05 m²
Solder Material: Solder
Solder Material Conductivity: 50 W/mK
Solder Thickness: 5e-05 m
Solder Resistance: 0.0125 °C/W

Copper Layer Material: Copper
Copper Layer Material Conductivity: 385 W/mK
Copper Layer Thickness: 3.5e-05 m
Copper Layer Resistance: 0.0011363636363636363 °C/W

PCB Thickness: 0.0006 m
Via Plating Thickness: 3.5e-05 m
Via Drill Diameter: 0.0003 m
Via Clearance: 0.00021213203435596422 m
Via Radious: 0.0003621320343559642 m
	Rows of Vias: 13
	Columns of Vias: 12
	Number of Vias: 156
	Area Utilization: 0.8858478249509544
Via Material: Copper
Via Material Conductivity: 385 W/mK
Via cross-section area: 1.7592918860102842e-08
Single Via Resistance: 88.58345626636103 °C/W
Array of Vias Resistance: 0.5678426683741091 °C/W

TIM Material: Mica
TIM Material Conductivity: 1.7 W/mK
TIM Thickness: 0.0002 m
TIM Resistance: 1